In [1]:
import pickle #read pickle file
import coffea
from topcoffea.modules.histEFT import HistEFT
from coffea.analysis_tools import PackedSelection
import topcoffea.modules.eft_helper as efth
import gzip #read zipped pickle file
import matplotlib.pyplot as plt #plot histograms
from matplotlib.backends.backend_pdf import PdfPages
import topcoffea.modules.utils as utils
import mplhep as hep
import numpy as np
import awkward as ak
np.seterr(divide='ignore', invalid='ignore', over='ignore')

import hist
from hist import Hist

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
NanoAODSchema.warn_missing_crossrefs = False

from mt2 import mt2
from coffea.nanoevents.methods import vector

In [69]:
from mt2 import mt2_arxiv

ModuleNotFoundError: No module named 'ROOT'

In [2]:
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20}
plt.rcParams.update(params)

In [3]:
# Clean the objects
def is_clean(obj_A, obj_B, drmin=0.4):
    objB_near, objB_DR = obj_A.nearest(obj_B, return_metric=True)
    mask = ak.fill_none(objB_DR > drmin, True)
    return (mask)

In [57]:
def make_mt2(l0, l1, met):
    nevents = len(np.zeros_like(met))
    misspart = ak.zip(
        {
            "pt": met.pt,
            "eta": 0,
            "phi": met.phi,
            "mass": np.full(nevents, 0),
        },
        with_name="PtEtaPhiMLorentzVector",
        behavior=vector.behavior,
    )

    mt2_val = mt2(
        l0.mass, l0.px, l0.py,                          # visible particle #1
        l1.mass, l1.px, l1.py,                          # visible particle #2 
        misspart.px, misspart.py,                       # missing transverse momentum
        np.zeros_like(met.pt), np.zeros_like(met.pt)    # invisible masses
    )

    return mt2_val

In [5]:
fname = "/cms/cephfs/data/store/user/hnelson2/mc/NanoGen/tW_April8/NanoGen_tW_new1/nanoGen_210.root"

In [6]:
# Load in events from root file
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TT01j2l_ref"},
).events()

In [7]:
wc_lst = utils.get_list_of_wc_names(fname)
print("wc list: ", wc_lst)

dataset = events.metadata['dataset']

wc list:  ['cHtbRe', 'ctGRe', 'ctGIm', 'cHQ3', 'cbWRe', 'cQl3', 'cleQt3Re', 'ctWRe', 'cleQt1Re']


In [8]:
# Extract the EFT quadratic coefficients and optionally use them to calculate the coefficients on the w**2 quartic function
# eft_coeffs is never Jagged so convert immediately to numpy for ease of use.
eft_coeffs = ak.to_numpy(events['EFTfitCoefficients']) if hasattr(events, "EFTfitCoefficients") else None

In [9]:
if eft_coeffs is None:
    event_weights = events["genWeight"]
else:
    event_weights = np.ones_like(events['event'])

In [15]:
######## Initialize objects  ########
genpart = events.GenPart
is_final_mask = genpart.hasFlags(["fromHardProcess","isLastCopy"])
ele  = genpart[is_final_mask & (abs(genpart.pdgId) == 11)]
mu   = genpart[is_final_mask & (abs(genpart.pdgId) == 13)]
nu_ele = genpart[is_final_mask & (abs(genpart.pdgId) == 12)]
nu_mu = genpart[is_final_mask & (abs(genpart.pdgId) == 14)]
nu = ak.concatenate([nu_ele,nu_mu],axis=1)
jets = events.GenJet

######## Lep selection  ########
e_selec = ((ele.pt>20) & (abs(ele.eta)<2.5))
m_selec = ((mu.pt>20) & (abs(mu.eta)<2.5))
leps = ak.concatenate([ele[e_selec],mu[m_selec]],axis=1)
leps = leps[ak.argsort(leps.pt, axis=-1, ascending=False)]

######## Jet selection  ########
jets = jets[(jets.pt>30) & (abs(jets.eta)<2.5)]
jets_clean = jets[is_clean(jets, leps, drmin=0.4) & is_clean(jets, nu, drmin=0.4)]

In [11]:
######## Event selections ########
nleps = ak.num(leps)
njets = ak.num(jets_clean)
at_least_two_leps = ak.fill_none(nleps>=2,False)
at_least_one_jet = ak.fill_none(njets>=1,False)

selections = PackedSelection()
selections.add('2l', at_least_two_leps)
selections.add('1j', at_least_one_jet)
event_selection_mask = selections.all('2l', '1j')

In [12]:
leps_cut = leps[event_selection_mask]
# print(leps_cut[:,0].E)
# print(leps_cut[:,0].pt)
# print(leps_cut[:,0].eta)
# print(leps_cut[:,0].phi)

In [44]:
leps_cut[:,1].py

<Array [16.5, -17.1, -170, ... 11.3, -18.4] type='4281 * float32'>

In [40]:
len(leps_cut[:,0][leps_cut[:,0].mass !=0])

0

In [62]:
met  = events.GenMET[event_selection_mask]
print(met.px)

nevents = len(np.zeros_like(met))
misspart = ak.zip(
    {
        "pt": met.pt,
        "eta": 0,
        "phi": met.phi,
        "mass": np.full(nevents, 0),
    },
    with_name="PtEtaPhiMLorentzVector",
    behavior=vector.behavior,
)

[80.2, 9.7, -21.6, 64.6, -33.7, 39.2, ... -91.5, 42.2, -58.9, -23.2, 28.2, 42.5]


In [65]:
print(f"{leps_cut[:,0][0].mass}, {leps_cut[:,0][0].px}, {leps_cut[:,0][0].py}, {leps_cut[:,1][0].mass}, {leps_cut[:,1][0].px}, {leps_cut[:,1][0].py}, {misspart[0].px}, {misspart[0].py}, 0, 0")

0.0, -62.19723453508825, 76.50274077938606, 0.0, 54.42772004730086, 16.505076623746838, 80.16275818329795, -69.01592790126294, 0, 0


In [68]:
mt2(0.0, -62.19723453508825, 76.50274077938606, 0.0, 54.42772004730086, 16.505076623746838, 80.16275818329795, -69.01592790126294, 0, 0, 0)

-1.3857048749923706

In [72]:
mt2_arxiv(0.0, -62.19723453508825, 76.50274077938606, 0.0, 54.42772004730086, 16.505076623746838, 80.16275818329795, -69.01592790126294, 0, 0)

63.60707182095828

In [60]:
len(met[met.pt>0])

4281

In [58]:
mt2_var = make_mt2(leps_cut[:,0], leps_cut[:,1], met)
print(mt2_var)

[-1.39, -1.39, -1.39, -1.39, -1.39, -1.39, ... -1.39, -1.39, -1.39, -1.39, -1.39]


In [16]:
proc_axis = hist.axis.StrCategory([], name="process", growth=True)
histos = {
    "mt2" : HistEFT(
                proc_axis, 
                hist.axis.Regular(bins=50, start=0, stop=500, name="mt2", label="$m_{T2}$ [GeV]"),
                wc_names=wc_lst, 
                label="Events"),
    "l0pt" : HistEFT(
                proc_axis, 
                hist.axis.Regular(bins=40, start=0, stop=400, name='l0pt', label='leading lepton $p_T$ [GeV]'),
                wc_names=wc_lst, 
                label="Events"),
}

In [22]:
histos['mt2'].fill(**{"mt2":mt2_var, 
                    "process":'tW', 
                    "weight": event_weights[event_selection_mask], 
                    "eft_coeff": eft_coeffs[event_selection_mask]})

In [23]:
histos['mt2'].as_hist({}).values()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [ ]:
njets_fill_info = {
    "njets": njets_cut, 
    "sample": dataset,
    "weight": event_weights[event_selection_mask], 
    "eft_coeff": eft_coeffs,
}

In [ ]:
histos["jet_flav"].fill(**jet_flav_fill_info)
histos["njets"].fill(**njets_fill_info)

In [ ]:
h = histos['njets']
# h.set_sm()
# h.set_wilson_coefficients(**couplings)
# print(h._wcs)
fig, ax = plt.subplots(1,1)
hist.plot1d(h, ax=ax, stack=False)
ax.legend()